In [1]:
!pip install transformers
!pip install emoji
!pip install datasets
!pip install tensorflow

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 31.4 MB/s 
     |████████████████████████████████| 895 kB 43.2 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 175 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=2988ce9042cbf4c950f3f48a6dd1054511ce91af88377056836af44070893a69
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 312 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 47.2 MB/s 
     |████████████████████████████████| 212 kB 54.1 MB/s 
     |████████████████████████████████| 134 kB 47.1 M

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)

model = TFAutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base",num_labels=2)


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/705M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from transformers import AutoTokenizer, DataCollatorWithPadding

from datasets import load_dataset

tweet_dataset = load_dataset("tweets_hate_speech_detection",split='train[:10%]')

Downloading:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/881 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset tweets_hate_speech_detection downloaded and prepared to /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2. Subsequent calls will reuse this data.


In [4]:
def tokenize_function(example):
    return tokenizer(example["tweet"], truncation=True)

tweet_dataset

Dataset({
    features: ['label', 'tweet'],
    num_rows: 3196
})

In [5]:
dataset_clean = tweet_dataset.train_test_split(train_size=0.8)

dataset_clean

DatasetDict({
    train: Dataset({
        features: ['label', 'tweet'],
        num_rows: 2556
    })
    test: Dataset({
        features: ['label', 'tweet'],
        num_rows: 640
    })
})

In [6]:
tokenized_datasets = dataset_clean.map(tokenize_function, batched=True)

tokenized_datasets

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2556
    })
    test: Dataset({
        features: ['label', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
})

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)


In [8]:

from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

In [11]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
opt = Adam(learning_rate=lr_scheduler)


loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=1)

319/319 [==============================] - 130s 349ms/step - loss: 0.1960 - accuracy: 0.9338 - val_loss: 0.1490 - val_accuracy: 0.9328


In [12]:
# !mkdir -p saved_model
# model.save('saved_model/my_model') 

In [13]:
# !zip -r /content/file.zip /content/saved_model

In [15]:
# from google.colab import files
# files.download("/content/file.zip")

In [16]:
model.evaluate(tf_validation_dataset)

80/80 [==============================] - 10s 124ms/step - loss: 0.1490 - accuracy: 0.9328


[0.14899970591068268, 0.932812511920929]